In [58]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, accuracy_score
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F
import pickle


# Constants
EEG_DATA_DIR = r'C:\Users\User\Documents\Lie detect data\TestData-1'
max_length = 500  # Define maximum length for padding

# Define EEGNet model
class EEGNet(nn.Module):
    def __init__(self, nb_classes, Chans=65, Samples=500, dropoutRate=0.5, 
                 kernLength=125, F1=8, D=2, F2=None, norm_rate=0.25, dropoutType='Dropout'):
        super(EEGNet, self).__init__()
        if F2 is None:
            F2 = F1 * D
        
        self.conv1 = nn.Conv2d(1, F1, (1, kernLength), padding='same', bias=False)
        self.batchnorm1 = nn.BatchNorm2d(F1)
        
        self.depthwiseConv = nn.Conv2d(F1, F1 * D, (Chans, 1), groups=F1, bias=False)
        self.batchnorm2 = nn.BatchNorm2d(F1 * D)
        
        self.averagePool1 = nn.AvgPool2d((1, 4))
        
        if dropoutType == 'SpatialDropout2D':
            self.dropout1 = nn.Dropout2d(dropoutRate)
        elif dropoutType == 'Dropout':
            self.dropout1 = nn.Dropout(dropoutRate)
        else:
            raise ValueError('dropoutType must be one of SpatialDropout2D or Dropout')
        
        self.separableConv1 = nn.Conv2d(F1 * D, F2, (1, 16), padding='same', bias=False)
        self.batchnorm3 = nn.BatchNorm2d(F2)
        
        self.averagePool2 = nn.AvgPool2d((1, 8))
        
        if dropoutType == 'SpatialDropout2D':
            self.dropout2 = nn.Dropout2d(dropoutRate)
        elif dropoutType == 'Dropout':
            self.dropout2 = nn.Dropout(dropoutRate)
        
        # Add more depth by adding another separable convolutional block
        self.separableConv2 = nn.Conv2d(F2, F2, (1, 16), padding='same', bias=False)
        self.batchnorm4 = nn.BatchNorm2d(F2)
        
        self.averagePool3 = nn.AvgPool2d((1, 8))
        
        if dropoutType == 'SpatialDropout2D':
            self.dropout3 = nn.Dropout2d(dropoutRate)
        elif dropoutType == 'Dropout':
            self.dropout3 = nn.Dropout(dropoutRate)
        
        self.flatten = nn.Flatten()
        self.dense = nn.Linear(F2 * (Samples // 256), nb_classes)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.batchnorm1(x)
        x = F.elu(x)
        
        x = self.depthwiseConv(x)
        x = self.batchnorm2(x)
        x = F.elu(x)
        x = self.averagePool1(x)
        x = self.dropout1(x)
        
        x = self.separableConv1(x)
        x = self.batchnorm3(x)
        x = F.elu(x)
        x = self.averagePool2(x)
        x = self.dropout2(x)
        
        # Pass through the additional depth layers
        x = self.separableConv2(x)
        x = self.batchnorm4(x)
        x = F.elu(x)
        x = self.averagePool3(x)
        x = self.dropout3(x)
        
        x = self.flatten(x)
        x = self.dense(x)
        x = self.softmax(x)
        
        return x
        
# Function to load and label data (same as in your training script)
def load_data(data_dir, max_length):
    X = []
    y = []
    file_list = os.listdir(data_dir)
    for file in file_list:
        with open(os.path.join(data_dir, file), 'rb') as f:
            data = pickle.load(f)
        label = 1 if 'truth' in file else 0
        if data.shape[1] > max_length:
            processed_data = data[:, :max_length]  # Cut data if it exceeds max_length
        else:
            processed_data = np.zeros((data.shape[0], max_length))
            processed_data[:, :data.shape[1]] = data  # Pad data if it is shorter than max_length
        X.append(processed_data)
        y.append(label)
    return np.array(X), np.array(y)

# Define dataset class
class EEGDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        # Ensure the data is reshaped to [1, Chans, Samples]
        return torch.tensor(self.X[idx], dtype=torch.float32).unsqueeze(0), torch.tensor(self.y[idx], dtype=torch.long)


# Main script
if __name__ == "__main__":
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load the saved model
    model_path = r'C:\Users\User\Documents\Lie detect data\Model\fold3_model_fold_4.pth'
    nb_classes = 2
    model = EEGNet(nb_classes=nb_classes).to(device)
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval

    # Move the model to the appropriate device
    model.to(device)

    # Load and preprocess the training data to create the scaler
    X, y = load_data(EEG_DATA_DIR, max_length)

    # Load the scaler
    with open(r'C:\Users\User\Documents\Lie detect data\Model\simpleEEGNet_scaler.pkl', 'rb') as f:
        scaler = pickle.load(f)

    X = scaler.transform(X.reshape(X.shape[0], -1))
    X = X.reshape(-1, 65, max_length)
    

    test_dataset = EEGDataset(X, y)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    all_labels = []
    all_predictions = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        print(f"batch shape: {X_batch.shape}")
        
        X_batch = X_batch.to(device)
        labels = y_batch.to(device)
        
        outputs = model(X_batch)
        _, predicted = torch.max(outputs.data, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Convert lists to numpy arrays for metric calculations
all_labels = np.array(all_labels)
all_predictions = np.array(all_predictions)

# Calculate metrics
accuracy = accuracy_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions)
recall = recall_score(all_labels, all_predictions)
f1 = f1_score(all_labels, all_predictions)
auc = roc_auc_score(all_labels, all_predictions)
conf_matrix = confusion_matrix(all_labels, all_predictions)


# Print the metrics
print(f"Accuracy: {accuracy:.4f}")
print(f'Precision: {precision}, Recall: {recall}, F1-score: {f1}, AUC: {auc}')
print('Confusion Matrix:')
print(conf_matrix)

    

batch shape: torch.Size([32, 1, 65, 500])
batch shape: torch.Size([32, 1, 65, 500])
batch shape: torch.Size([20, 1, 65, 500])
Accuracy: 0.4405
Precision: 0.5777777777777777, Recall: 0.48148148148148145, F1-score: 0.5252525252525253, AUC: 0.42407407407407405
Confusion Matrix:
[[11 19]
 [28 26]]
